In [1]:
import json
import jsonlines
import statistics
from tqdm import tqdm
import openai
openai.api_key = ''

In [2]:
with open('inference_data.json','r') as f: #inference_data.json
    wiki = json.load(f)

In [7]:
wiki[0]

{'title': 'Czechs in Argentina',
 'answer': ['Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.',
  'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.',
  'Argentina has the largest Czech community in Latin America.',
  'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.',
  'There are four Czech immigration periods in Argentina.',
  'The first one is slightly before to the World War 1, the second one from 1920 to 1930, the third one during the World War 2 and the fourth one, the smallest in migration proportion, during 1990. During the first two periods, the immigration group was mainly integrated by workers and farmers motivated by economic reasons.']}

In [8]:
print(len(wiki))

113678


In [9]:
num = 100

In [10]:
###persona 부여
## 1: 유치원생이 질문하는 듯한 느낌
## 2: 교수님께 질문하는 공손한

instruction = f"You are an automatic assistant that generates appropriate question based on the predefined answer.\n"
## 유치원생이 질문하는 듯한 느낌
#persona = "Generate questions with a tone as if a kindergarten student is asking.\n"
## 학생이 교수님께 질문하는 듯한 느낌
persona = "Generate questions with the tone as if a student is asking a professor.\n"
WikiDialog_LLM_instruction = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = instruction + persona + base + tmp + QA_pair
        #print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_instruction.append(current_dialog)

100%|██████████| 100/100 [26:59<00:00, 16.19s/it]  


In [11]:
with open(f'WikiDialog_LLM_instruction_student_to_professor.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_instruction, json_file, indent=2)

In [28]:
with open('WikiDialog_LLM_instruction_kindergarten.json_scores.json','r') as f: #inference_data.json
    kinder = json.load(f)
with open('WikiDialog_LLM_instruction_polite_student_to_professor.json_scores.json','r') as f: #inference_data.json
    polite = json.load(f)
with open('WikiDialog_LLM_instruction_professor_to_student.json_scores.json','r') as f: #inference_data.json
    professor = json.load(f)
with open('WikiDialog_LLM_instruction_student_to_professor.json_scores.json','r') as f: #inference_data.json
    student = json.load(f)

In [3]:
ls

 231203_dialogsystem_project_edit-Copy1.ipynb
 231203_dialogsystem_project_edit-Copy2.ipynb
 231203_dialogsystem_project_edit.ipynb
 231203_dialogsystem_project_rquge_scores.ipynb
 RQUGE_Score.sh
 WikiDialog_LLM_instruction_kindergarten.json
 WikiDialog_LLM_instruction_kindergarten.json_scores.json
 WikiDialog_LLM_instruction_polite_student_to_professor.json
 WikiDialog_LLM_instruction_polite_student_to_professor.json_scores.json
 WikiDialog_LLM_instruction_professor_to_student.json
 WikiDialog_LLM_instruction_professor_to_student.json_scores.json
 WikiDialog_LLM_instruction_student_to_professor.json
 WikiDialog_LLM_instruction_student_to_professor.json_scores.json
'datasets2023-12-03 02:57:04.json'
 inference_data.json
 old/
 output.txt
 quip-512-mocha/
 run_RQUGE.py


In [4]:
#dialogue_project/old/WikiDialog_LLM_instruction_100_scores.json
#dialogue_project/old/WikiDialog_LLM_instruction_passage_title_100_scores.json

with open('old/WikiDialog_LLM_instruction_passage_title_100_scores.json','r') as f: #inference_data.json
    title = json.load(f)
    
with open('old/WikiDialog_LLM_instruction_100_scores.json','r') as f: #inference_data.json
    instruction = json.load(f)

In [5]:
average_score=dict()
persona_type=['title','instruction']
for persona in persona_type:
    average_score[persona]=[]

In [6]:
for i in range(len(title)):
    temp=0
    for j in range(len(title[i])):
        temp+=title[i][j]['score']
    average_score['title'].append(temp/len(title[i]))

In [7]:
for i in range(len(instruction)):
    temp=0
    for j in range(len(instruction[i])):
        temp+=instruction[i][j]['score']
    average_score['instruction'].append(temp/len(instruction[i]))

In [27]:
num=84 
title[num]

[{'question': 'Who is Samuel Der-Yeghiayan?',
  'answer': 'Samuel Der-Yeghiayan Samuel Der-Yeghiayan (born February 16, 1952) is a former United States District Judge of the United States District Court for the Northern District of Illinois.',
  'score': 4.641402244567871},
 {'question': "What is noteworthy about Samuel Der-Yeghiayan's appointment as a federal judge?",
  'answer': 'Confirmed in 2003, he is noteworthy for being the first Armenian immigrant federal judge in the United States.',
  'score': 4.020795822143555},
 {'question': 'When did Samuel Der-Yeghiayan retire and where was he born and raised?',
  'answer': 'He retired from active service in 2018. Der-Yeghiayan was born in Aleppo, Syria to Armenian parents and raised in Beirut, Lebanon.',
  'score': 2.046229839324951},
 {'question': 'What age was Samuel Der-Yeghiayan when he moved to the United States?',
  'answer': 'He moved to the United States at age 19.',
  'score': 4.914449214935303},
 {'question': 'Where did Samuel 

In [28]:
wiki[num]['title'],instruction[num]

('Samuel Der-Yeghiayan',
 [{'question': 'Who is Samuel Der-Yeghiayan?',
   'answer': 'Samuel Der-Yeghiayan Samuel Der-Yeghiayan (born February 16, 1952) is a former United States District Judge of the United States District Court for the Northern District of Illinois.',
   'score': 4.641402244567871},
  {'question': 'When was he confirmed and what is he particularly known for?',
   'answer': 'Confirmed in 2003, he is noteworthy for being the first Armenian immigrant federal judge in the United States.',
   'score': 4.321036338806152},
  {'question': 'What is his background and when did he retire from his service?',
   'answer': 'He retired from active service in 2018. Der-Yeghiayan was born in Aleppo, Syria to Armenian parents and raised in Beirut, Lebanon.',
   'score': 4.077038288116455},
  {'question': 'At what age did he move to the United States?',
   'answer': 'He moved to the United States at age 19.',
   'score': 4.925430774688721},
  {'question': 'Where did he receive his educ

In [ ]:
instruction[0]

In [10]:
difference_max=0
index=0
temp=[]
print("title, instruction")
for i in range(len(average_score['title'])):
    
    
    
    #temp.append([average_score['student'][i],average_score['polite'][i],average_score['kinder'][i],average_score['professor'][i]])
    print(i,average_score['title'][i],average_score['instruction'][i])

title, instruction
0 4.241254210472107 3.6870089371999106
1 1.8522308468818665 3.5979541540145874
2 4.195961236953735 4.028608163197835
3 3.8360143105189004 3.4986937443415322
4 3.5501325527826944 3.337218721707662
5 3.732837518056234 3.644347310066223
6 3.658749580383301 4.025002121925354
7 3.2423744996388755 4.107247948646545
8 2.050810217857361 2.920435070991516
9 3.8894114891688027 3.023757815361023
10 2.9635319908459983 4.056617021560669
11 3.117670774459839 3.4991023540496826
12 4.162436326344808 4.252162019411723
13 3.8448652029037476 3.847372591495514
14 3.6374402046203613 4.229695439338684
15 2.8294990062713623 2.5072377622127533
16 3.782657027244568 4.693100929260254
17 3.6338963905970254 3.7476298014322915
18 4.532158454259236 4.253901282946269
19 3.8601222038269043 4.110776305198669
20 3.835901896158854 4.372199932734172
21 3.798807978630066 3.533073822657267
22 3.2839154998461404 3.983028789361318
23 3.3469309409459433 3.5929301182428994
24 3.1549574931462607 3.30166697502

In [31]:
average_score=dict()


persona_type=['kinder','polite','professor','student']
for persona in persona_type:
    average_score[persona]=[]
   

In [35]:
for i in range(len(student)):
    temp=0
    for j in range(len(student[i])):
        temp+=student[i][j]['score']
    average_score['student'].append(temp/len(student[i]))


In [49]:
difference_max=0
index=0
temp=[]
print("polite, student, kinder, professor")
for i in range(len(average_score['student'])):
    
    
    
    #temp.append([average_score['student'][i],average_score['polite'][i],average_score['kinder'][i],average_score['professor'][i]])
    print(i,[average_score['polite'][i],average_score['student'][i],average_score['kinder'][i],average_score['professor'][i]])



polite, student, kinder, professor
0 [3.6567852099736533, 3.346644560496012, 3.3304516275723777, 3.6202032566070557]
1 [2.258514702320099, 3.1596302588780723, 3.5484630266825357, 2.385486344496409]
2 [3.0924575527509055, 2.8189639151096344, 3.864801287651062, 3.17166539033254]
3 [2.316944142182668, 2.620239873727163, 3.3634960651397705, 2.652308762073517]
4 [3.0677186449368796, 3.144486387570699, 3.2209473848342896, 2.8938809633255005]
5 [3.0849194526672363, 2.4531315565109253, 3.6661837100982666, 2.545265873273214]
6 [2.328707456588745, 3.473642349243164, 3.28950027624766, 3.4046090046564736]
7 [2.0325583020846048, 3.029031674067179, 2.3141767978668213, 3.512774646282196]
8 [2.3927407264709473, 2.5259755849838257, 2.607531189918518, 2.039141595363617]
9 [2.7300833066304526, 3.358702619870504, 3.47654390335083, 3.410926600297292]
10 [4.033601522445679, 2.7372705340385437, 3.914924065272013, 3.1009623805681863]
11 [3.8956568241119385, 4.241907119750977, 3.0502870082855225, 3.44354033470

In [45]:
average_score['student'][29],average_score['kinder'][29]

(2.385336180528005, 4.02540119489034)

In [46]:
student[29],kinder[29],wiki[29]

([{'question': "Can you tell me who else was part of the 1987 All Japan Women's rookie class along with Toshiyo Yamada?",
   'answer': "Toshiyo Yamada Toshiyo Yamada part of the 1987 All Japan Women's rookie class which also featured Manami Toyota, Etsuko Mita and Mima Shimoda.",
   'score': 1.9728389978408813},
  {'question': 'Do you know when and against whom Toshiyo Yamada made her debut?',
   'answer': 'Yamada debuted on July 27, 1987 against Chiaki Ichikawa.',
   'score': 1.4990234375},
  {'question': 'Could you tell me about any awards or recognitions Toshiyo Yamada received in her rookie year?',
   'answer': "Yamada was awarded the 1987 Rookie of the Year award by All Japan Women's.",
   'score': 1.6843276023864746},
  {'question': 'Can you provide information on any championships Toshiyo Yamada won early in her career?',
   'answer': 'On July 19, 1988, she captured the AJW Junior Championship in a tournament final against Miori Kamiya.',
   'score': 2.795415163040161},
  {'ques

# base prompt 사용

In [6]:

WikiDialog_LLM = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]

        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = base + tmp + QA_pair
#         print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        tmp = tmp +  f"\nA: {question}\n B: {answer}"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM.append(current_dialog)

100%|██████████| 100/100 [40:26<00:00, 24.27s/it]  


In [7]:
print(base)

        Please fill in only [BLANK] in the next dialogue.
         [START]
 


In [ ]:
WikiDialog_LLM

# with instruction

In [14]:
instruction = f"You are an automatic assistant that generates appropriate question based on the predefined answer.\n"

In [15]:

WikiDialog_LLM_instruction = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = instruction + base + tmp + QA_pair
        #print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_instruction.append(current_dialog)

100%|██████████| 100/100 [22:47<00:00, 13.68s/it]  


In [ ]:
WikiDialog_LLM_instruction

# with instruction & passage title

In [ ]:

WikiDialog_LLM_instruction_passage_title = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    dialog_topic = wiki[i]['title']
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        title_hint = f"The conversation is about {dialog_topic}.\n"
        
        
        PROMPT = instruction + title_hint + base + tmp + QA_pair
        print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content 
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_instruction_passage_title.append(current_dialog)

# One shot 

In [32]:
# sample = """
# 'You are an automatic assistant that generates appropriate question based on the predefined answer.
# The conversation is about Arthur F. Hebard.
# Please fill in only [BLANK] in the next dialogue.
# [START] 
# A: Who is Arthur F. Hebard?
# B: Arthur F. Hebard  Arthur Foster Hebard (born 2 March 1940) is Distinguished Professor of Physics at University of Florida in Gainesville, Florida.
# A: What is Arthur F. Hebard particularly noted for?
# B: He is particularly noted for leading the discovery of superconductivity in Buckminsterfullerene in 1991.
# A: Where did Arthur F. Hebard receive his education?
# B: Art Hebard attended The Hotchkiss School and graduated with a BA in Physics from Yale University in 1962.
# A: Who was Arthur F. Hebard\'s PhD advisor and what was his thesis about?
# B: He obtained his PhD from Stanford University in 1971 under William M. Fairbank with thesis "Search for fractional charge using low temperature techniques".
# A: What was Arthur F. Hebard\'s career path after completing his PhD?
# B: After a spell as a Research Associate at Stanford, he became a member of the Technical Staff at AT&T Bell Telephone Laboratories.
# A: [BLANK]
# B: He moved to the University of Florida as a Professor in 1996, and in 2007 was given the title of Distinguished Professor.
# [END]'
# """

In [39]:
shot_1 = """
Please fill in only [BLANK] in the next dialogue.
[START] 
A: Who is Michael Jordan?
B: Michael Jeffrey Jordan also known by his initials MJ, is an American businessman and former professional basketball player. 
A: Is Michael Jordan the greatest player?
B: His profile on the official National Basketball Association (NBA) website states that 'by acclamation, Michael Jordan is the greatest basketball player of all time.'
A: How many games did Michael Jordan play and won?
B: He played fifteen seasons in the NBA, winning six NBA championships with the Chicago Bulls. 
A: [BLANK]
B: He was integral in popularizing the sport of basketball and the NBA around the world in the 1980s and 1990s, becoming a global cultural icon.
[END]
How could Michael Jordan be described in terms of global influence?


"""

In [42]:

WikiDialog_LLM_shot = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        
        shot = shot_1
        
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = shot + base + tmp + QA_pair
        #print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_shot.append(current_dialog)

100%|██████████| 100/100 [11:53<00:00,  7.14s/it]


In [41]:
WikiDialog_LLM_shot

[[{'question': 'What can you tell me about Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the significance of the Czech community in Argentina?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'Can you tell me more about these four periods of Czech immigration?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'What were the main reasons for Czech immigration during these periods?',
   'answer': 'The first one is slightly before to the World War 1, the secon

# Save json

In [18]:
WikiDialog_LLM[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the status of the Czech community in Argentina today?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'What areas did the Czechs primarily settle in?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you tell me more about the reasons for immigration during the third and fourth periods?',
   'answer': 'The first one is s

In [19]:
WikiDialog_LLM_instruction[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina by 1970?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'Which Latin American country has the largest Czech community?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the majority of Czechs settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you describe the characteristics of the immigrants during the first two periods of Czech immigration in Arg

In [20]:
WikiDialog_LLM_instruction_passage_title[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina by 1970?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'Which Latin American country has the largest Czech community?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'Can you elaborate on the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'What were the characteristics of each of these immigration periods?',
   'answer': 'The first one is sli

In [43]:
WikiDialog_LLM_shot[:2]

[[{'question': 'What can you tell me about Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the significance of the Czech community in Argentina?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you tell me more about each period of Czech immigration?',
   'answer': 'The first one is slightly before to the World War 1, the second one from 1920 t

In [21]:
print(len(WikiDialog_LLM))
print(len(WikiDialog_LLM_instruction))
print(len(WikiDialog_LLM_instruction_passage_title))
print(len(WikiDialog_LLM_shot))

100
100
100


In [25]:
with open(f'WikiDialog_LLM_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM, json_file, indent=2)

In [26]:
with open(f'WikiDialog_LLM_instruction_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_instruction, json_file, indent=2)

In [27]:
with open(f'WikiDialog_LLM_instruction_passage_title_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_instruction_passage_title, json_file, indent=2)

In [44]:
with open(f'WikiDialog_LLM_shot_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_shot, json_file, indent=2)